In [1]:
import pprint
import json
import pathlib
import pandas as pd
import os
import importlib
import networkx as nx
import matplotlib.pyplot as plt
import math
from random import sample
from collections import Counter

In [2]:
from libWill.helpers import *
# importlib.reload(helpers)
pp = pprint.PrettyPrinter(indent=4)

In [3]:
dirty_hourly = pathlib.Path('../../../data/dirty/hourly/')
jsonl_files = os.listdir(dirty_hourly)

In [4]:
all_tweets = []

for file in jsonl_files:
    all_tweets.extend(filter_tweet_fields_as_dict(pathlib.Path(dirty_hourly,file)))

In [5]:
df = pd.DataFrame(all_tweets[0:20000])
df.tail()

,created_at,screen_name,full_text,geo,id,lang,retweet_count,rt_status_screen_name,rt_status_id,mentions,mention_ids
19995,Thu Oct 24 01:37:08 +0000 2019,gpscryptoq,RT @Etherium18: I will choose 20 random follow...,None,1187181196481986560,en,0,Etherium18,1123081089713487878,[Etherium18],[1123081089713487878]
19996,Thu Oct 24 01:37:07 +0000 2019,bitcoinincoins,Goal Setting Software For High Achievers) \n@B...,None,1187181192841445376,en,0,NaN,NaN,"[bitcoinincoins, InvestCrypForex]","[924014696285523971, 1155780138518437888]"
19997,Thu Oct 24 01:37:05 +0000 2019,bitcoinincoins,Automated Forex Tools - Forex Robots - Expert ...,None,1187181182628368385,en,0,NaN,NaN,"[bitcoinincoins, InvestCrypForex]","[924014696285523971, 1155780138518437888]"
19998,Thu Oct 24 01:37:03 +0000 2019,MarcusTyZI86,RT @Trezor: From now on we’ll be releasing two...,None,1187181173673463810,en,0,Trezor,1431204614,[Trezor],[1431204614]
19999,Thu Oct 24 01:37:00 +0000 2019,ELSULLY23,@zerohedge Gold and bitcoin should be pumping ...,None,1187181160566149121,en,0,NaN,NaN,[zerohedge],[18856867]


In [6]:
df = df[['created_at','screen_name','full_text','id','geo','lang','mention_ids','mentions','retweet_count','rt_status_id','rt_status_screen_name']]

In [7]:
df.head(10)

,created_at,screen_name,full_text,id,geo,lang,mention_ids,mentions,retweet_count,rt_status_id,rt_status_screen_name
0,Tue Oct 29 11:58:59 +0000 2019,luisgfagundes,RT @CCNMarkets: Starbucks Will Accept Bitcoin ...,1189149629977452544,None,en,[1856523530],[CCNMarkets],0,1856523530,CCNMarkets
1,Tue Oct 29 11:58:59 +0000 2019,thefinancialfor,"RT @thefinancialfor: Every year since 2015, #B...",1189149627469193216,None,en,[2557010554],[thefinancialfor],0,2557010554,thefinancialfor
2,Tue Oct 29 11:58:57 +0000 2019,opp4wealth,RT @RedditBTC: This perfectly explains the cur...,1189149619713957888,None,en,[951006367],[RedditBTC],0,951006367,RedditBTC
3,Tue Oct 29 11:58:56 +0000 2019,Techtelegraph4,Russian Metals Plant to Start Bitcoin Mining h...,1189149616459190272,None,en,[],[],0,NaN,NaN
4,Tue Oct 29 11:58:52 +0000 2019,tiago60334177,RT @BitcollarDApp: 3 days to go in IEO phase o...,1189149599581265920,None,en,[1140825020463104000],[BitcollarDApp],0,1140825020463104000,BitcollarDApp
5,Tue Oct 29 11:58:49 +0000 2019,CryptoCryptoNe3,Speculation Abounds Over the Cause of Bitcoin'...,1189149586788618240,None,en,[],[],0,NaN,NaN
6,Tue Oct 29 11:58:46 +0000 2019,Vindyne8,RT @BTCTN: Why Central Banks Are Not Designed ...,1189149572259553281,None,en,[3367334171],[BTCTN],0,3367334171,BTCTN
7,Tue Oct 29 11:58:44 +0000 2019,IM2LOWKEY,RT @CCNMarkets: Starbucks Will Accept Bitcoin ...,1189149566395998209,None,en,[1856523530],[CCNMarkets],0,1856523530,CCNMarkets
8,Tue Oct 29 11:58:41 +0000 2019,el_priist,@HuobiGlobal $bitcoin $CasinoCoin,1189149554639282178,None,en,[914029581610377217],[HuobiGlobal],0,NaN,NaN
9,Tue Oct 29 11:58:41 +0000 2019,AvyLorenCohen,There are 3 million #Bitcoins left. Why does t...,1189149554370842629,None,en,[],[],0,NaN,NaN


In [8]:
df_network = df[['screen_name','mentions','rt_status_screen_name']]
df_network.head()

,screen_name,mentions,rt_status_screen_name
0,luisgfagundes,[CCNMarkets],CCNMarkets
1,thefinancialfor,[thefinancialfor],thefinancialfor
2,opp4wealth,[RedditBTC],RedditBTC
3,Techtelegraph4,[],NaN
4,tiago60334177,[BitcollarDApp],BitcollarDApp


In [9]:
smallnet = df_network
smallnet


,screen_name,mentions,rt_status_screen_name
0,luisgfagundes,[CCNMarkets],CCNMarkets
1,thefinancialfor,[thefinancialfor],thefinancialfor
2,opp4wealth,[RedditBTC],RedditBTC
3,Techtelegraph4,[],NaN
4,tiago60334177,[BitcollarDApp],BitcollarDApp
...,...,...,...
19995,gpscryptoq,[Etherium18],Etherium18
19996,bitcoinincoins,"[bitcoinincoins, InvestCrypForex]",NaN
19997,bitcoinincoins,"[bitcoinincoins, InvestCrypForex]",NaN
19998,MarcusTyZI86,[Trezor],Trezor


##Unpacking the mentions column to have multiple rows for each item in the mentions list

In [10]:
smallnet_exploded = smallnet.explode('mentions')

In [11]:
smallnet_exploded.reset_index(inplace=True,drop=True)

In [12]:
smallnet_exploded.tail()

,screen_name,mentions,rt_status_screen_name
25691,bitcoinincoins,InvestCrypForex,NaN
25692,bitcoinincoins,bitcoinincoins,NaN
25693,bitcoinincoins,InvestCrypForex,NaN
25694,MarcusTyZI86,Trezor,Trezor
25695,ELSULLY23,zerohedge,NaN


In [13]:
edges = list(zip(smallnet_exploded['screen_name'],smallnet_exploded['mentions']))

In [14]:
edges = [t for t in edges if not any(isinstance(n, float) and math.isnan(n) for n in t)]

In [15]:
#print(edges)

In [16]:
twit_graph = nx.Graph()

In [17]:
twit_graph.add_nodes_from(smallnet['screen_name'])

In [18]:
twit_graph.add_edges_from(edges)

In [19]:
#list(twit_graph.nodes)

In [20]:
#nx.draw(twit_graph)

In [21]:
nx.density(twit_graph)

0.00013969506691264492

In [22]:
nx.number_of_nodes(twit_graph)

15339

In [23]:
nx.number_of_edges(twit_graph)

16433

In [24]:
drop_nodes = []
for i in twit_graph.nodes:
    if twit_graph.degree(i) < 5:
        drop_nodes.append(i)

In [25]:
twit_graph.remove_nodes_from(drop_nodes)

In [26]:
nx.number_of_nodes(twit_graph)

1259

In [27]:
import community

In [28]:
parts = community.best_partition(twit_graph)
values = [parts.get(node) for node in twit_graph.nodes()]

AttributeError: module 'community' has no attribute 'best_partition'

In [ ]:
node_list = list(twit_graph.nodes())
community_dict = { n:c for n,c in zip(node_list,values)}
nx.set_node_attributes(twit_graph, community_dict, 'community')

In [ ]:
from bokeh.io import show, save, output_file 
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral8
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
import holoviews as hv


In [ ]:
hv.extension('bokeh')

In [ ]:
%opts Graph [width=800 height=800]
padding = dict(x=(-1.1, 1.1), y=(-1.1, 1.1))
twitter_graph = hv.Graph.from_networkx(twit_graph, nx.layout.spring_layout)\
                .redim.range(**padding)\
                .options(color_index='community', cmap='Category20',node_size=7, edge_line_width=1,edge_line_color='#d3d3d3',)
hv.save(twitter_graph, 'holoview_test.html')                                        
#hv.save(Graph, 'holoview_test.html')

In [ ]:
twitter_graph

In [ ]:
# #hover = HoverTool(tooltips=[('screen_name', twit_graph.nodes)])
                  
# plot = figure(title="5000 Tweets Test",
#               width=800,height=600,
#               tools=['box_zoom','pan','wheel_zoom','reset'], x_range=(-1.1,1.1), y_range=(-1.1,1.1))

# graph = from_networkx(twit_graph, nx.spring_layout, scale=3, center=(0,0))
# # graph.node_renderer.data_source = values

# graph.node_renderer.data_source.data['community'] = [i[1]['community'] for i in twit_graph.nodes(data=True)]
# graph.node_renderer.glyph = Circle(
#     size=10, 
#     fill_color=linear_cmap('community', 'Spectral8', 0, len(twit_graph.nodes()))
# )

# plot.renderers.append(graph)

# #output_file("5000_tweet_test_community.html")
# show(plot)

In [ ]:
from holoviews.operation.datashader import datashade, bundle_graph


In [ ]:
bundled = bundle_graph(twitter_graph)
bundled

:Graph   [start,end]

In [433]:
sorted(twit_graph.degree, key=lambda x: x[1], reverse=True)

[('vgorefs', 41),
 ('StakeFlip', 41),
 ('satasion', 17),
 ('_blockandchain_', 16),
 ('M_ednut', 12),
 ('donlekside', 9),
 ('_NicolasCantu', 9),
 ('aluko_wemimo', 9),
 ('sumawun', 9),
 ('Lionel46294550', 8),
 ('lopp', 8),
 ('Jaygold100', 7),
 ('APompliano', 7),
 ('aantonop', 6),
 ('bitsko_xt', 5),
 ('BitcoinGent', 5),
 ('pSN8MaqQCsJlBK7', 5),
 ('HappyChase02', 5),
 ('thibm_', 5),
 ('cantonrd', 5),
 ('_JustinMoon_', 5),
 ('akalayci34', 5),
 ('CryptoTownEU', 5),
 ('Rhythmtrader', 5),
 ('Bitboy_Crypto', 5),
 ('chrisjpn8', 4),
 ('huron39412249', 4),
 ('kolskols', 4),
 ('admofchromia', 4),
 ('x_decr3', 4),
 ('GagneLegal', 4),
 ('ribeetperogamer', 4),
 ('DesiCryptoHodlr', 4),
 ('Capfolio', 4),
 ('Blockchayne', 4),
 ('CoinEmergency', 4),
 ('zees_grillz', 4),
 ('szenpapi', 3),
 ('BitcoinTina', 3),
 ('sarkarnilesh215', 3),
 ('microminer1234', 3),
 ('LoanSh4rk', 3),
 ('Mel_1104', 3),
 ('richard00295204', 3),
 ('1001010c', 3),
 ('CryptoCinematic', 3),
 ('EntangleIT', 3),
 ('DyubsE', 3),
 ('newstwi

In [163]:
community = nx.get_node_attributes(twit_graph,'Community')

In [175]:
largest_community = Counter(community['Bitcoin'])
largest_community.most_common()

[(8, 97),
 (30, 94),
 (7, 84),
 (5, 75),
 (112, 72),
 (0, 71),
 (3, 70),
 (27, 67),
 (60, 67),
 (21, 62),
 (24, 62),
 (35, 58),
 (57, 56),
 (31, 52),
 (9, 50),
 (14, 48),
 (205, 43),
 (12, 40),
 (33, 35),
 (74, 31),
 (11, 25),
 (278, 23),
 (22, 22),
 (37, 19),
 (64, 17),
 (130, 16),
 (1, 14),
 (15, 12),
 (18, 12),
 (125, 12),
 (56, 10),
 (114, 9),
 (174, 9),
 (299, 9),
 (2, 8),
 (50, 8),
 (13, 7),
 (29, 7),
 (49, 7),
 (68, 7),
 (106, 7),
 (163, 7),
 (190, 7),
 (281, 7),
 (372, 7),
 (23, 6),
 (48, 6),
 (66, 6),
 (71, 6),
 (72, 6),
 (113, 6),
 (135, 6),
 (176, 6),
 (244, 6),
 (266, 6),
 (331, 6),
 (19, 5),
 (20, 5),
 (26, 5),
 (38, 5),
 (42, 5),
 (46, 5),
 (53, 5),
 (54, 5),
 (69, 5),
 (93, 5),
 (103, 5),
 (120, 5),
 (122, 5),
 (133, 5),
 (151, 5),
 (154, 5),
 (184, 5),
 (294, 5),
 (459, 5),
 (4, 4),
 (16, 4),
 (36, 4),
 (39, 4),
 (45, 4),
 (67, 4),
 (83, 4),
 (85, 4),
 (91, 4),
 (126, 4),
 (138, 4),
 (139, 4),
 (140, 4),
 (149, 4),
 (159, 4),
 (167, 4),
 (180, 4),
 (197, 4),
 (204, 4),


In [178]:
print(len(community['Bitcoin']))

2632


In [176]:
nodes = list(twit_graph.nodes)

In [182]:
print(nodes[3])

Techcenterr


In [36]:
isolates = nx.isolates(twit_graph)

In [38]:
list(isolates)

['BtcEthereum',
 'RockstaTrading',
 'squatch_crypto',
 'FinTechNames',
 'PatriotCrypto',
 'BitBullCapital',
 'duniajalanjalan',
 'Etherbankingita',
 'Franzferdinan57',
 'ICOResearchGRP',
 'SportsbookBTC',
 'BitcoinCatz',
 'Crypt0Constella',
 'VoCtpgC8Ki241Rg',
 'KriptoKanalim',
 'webnowcompany',
 'coinbureau',
 'mospretmen',
 'ALTcoin_Ninjas',
 'bitlyfool',
 'retiredin2016',
 'HashDeploy',
 'TheeSeanH',
 'Lobaoal',
 'ShillHater',
 'ib_cpa',
 'AntonioSerbia',
 'SnowflakesAwwww',
 'Blacktradelines',
 'cryptogarten',
 'crypto__mak',
 'jadwahab',
 'HydaxE',
 'coinsonnews',
 'BizSoftwareEdu',
 'thecryptorep',
 'Ursalacher',
 'AinslieWealth',
 'TheSh0tC4ll3r',
 'InvestForward18',
 'icoinmarket1',
 'dragonwolftech',
 'robertdolci',
 'NachoThome',
 'DarkNodeGuru',
 'foxesden34',
 'BigCheds',
 'help_me_plzplz_',
 'News_1jl4',
 'Teokanistras',
 'PaulESamson',
 'AvyLorenCohen',
 'grbroker',
 'Cryptonetwealth',
 'GetCoinJar',
 'URBitcoinwear',
 'blockchainoshi',
 'bitcoin_jar',
 'FreeCryptoDeals',

In [71]:
twit_graph.remove_nodes_from(list(nx.isolates(twit_graph)))

In [72]:
nx.number_of_nodes(twit_graph)

3156

[('APompliano', 84),
 ('Rhythmtrader', 54),
 ('Cointelegraph', 38),
 ('jack', 29),
 ('100trillionUSD', 26),
 ('Bitcoin', 22),
 ('PeterSchiff', 21),
 ('qhfofficial', 20),
 ('BitCoin_Invest_', 20),
 ('Poetry4Bitcoin', 17),
 ('coindesk', 16),
 ('Hermetec', 16),
 ('Satoshis_D', 15),
 ('SammarA12281909', 13),
 ('Bitcoin_Beyond', 13),
 ('codeyisfun', 12),
 ('CalvinAyre', 11),
 ('TheCryptoconomy', 11),
 ('popesta', 11),
 ('realDonaldTrump', 11),
 ('Bakkt', 11),
 ('PeterMcCormack', 11),
 ('american_hodl', 11),
 ('CryptoOz', 10),
 ('Hanakookie1', 10),
 ('NemesisBSV', 10),
 ('Cripto_Bitcoin', 10),
 ('BTCTN', 10),
 ('StormIsUponUs', 10),
 ('ibmlahamad', 9),
 ('BitcoinEcon', 9),
 ('Coinicarus', 9),
 ('YobitsVision', 9),
 ('LomahCrypto', 9),
 ('Mihalbig', 9),
 ('Xentagz', 9),
 ('MZietzke', 9),
 ('Anders_', 9),
 ('trylolli', 9),
 ('HotepJesus', 9),
 ('HonorAfricans', 9),
 ('theonevortex', 8),
 ('InvestCrypForex', 8),
 ('Emperor_YZ', 8),
 ('channelBSV', 8),
 ('mindstatex', 8),
 ('pierre_rochard', 8),

In [85]:
nx.average_degree_connectivity(twit_graph)

{1: 6.369631236442516,
 3: 3.8732943469785575,
 2: 7.258947368421053,
 4: 4.375,
 9: 4.981481481481482,
 6: 2.8015873015873014,
 8: 3.46875,
 5: 2.9348837209302325,
 7: 4.333333333333333,
 84: 3.0357142857142856,
 11: 3.9220779220779223,
 10: 3.8333333333333335,
 13: 6.230769230769231,
 16: 2.875,
 15: 4.8,
 21: 6.761904761904762,
 17: 5.823529411764706,
 38: 1.9210526315789473,
 54: 1.5740740740740742,
 29: 2.0,
 22: 3.3181818181818183,
 20: 1.025,
 12: 1.8333333333333333,
 26: 2.923076923076923}

In [93]:
edges_for_node_x = twit_graph.degree('Rhythmtrader')

In [94]:
print(edges_for_node_x)

54


In [100]:
print(len(drop_nodes))

2780
